In [2]:
import os
os.environ["JAX_PLATFORMS"] = "cpu"

import jax
import jax.numpy as jnp
import jmp


half = jnp.float16  # On TPU this should be jnp.bfloat16.
full = jnp.float32

global_list = []

def log2(x):
  global_list.append(x)
  ln_x = jnp.log(x)
  ln_2 = jnp.log(2.0)
  return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0))


# Our policy specifies that we will store parameters in full precision but will
# compute and return output in half precision.
low_policy = jmp.Policy(compute_dtype=half,
                       param_dtype=half,
                       output_dtype=half)

high_policy = jmp.Policy(compute_dtype=full,
                       param_dtype=full,
                       output_dtype=full)

def layer(params, x):
  params, x = low_policy.cast_to_compute((params, x))
  w1, b1, w2, b2 = params
  y = x @ w1 + b1

  y, w2, b2 = high_policy.cast_to_compute((y, w2, b2))
  z = y @ w2 + b2
  return high_policy.cast_to_output(z)

params = jnp.ones([20, 30], dtype=low_policy.param_dtype), jnp.ones([30], dtype=low_policy.param_dtype), jnp.ones([30, 40], dtype=low_policy.param_dtype), jnp.ones([40], dtype=low_policy.param_dtype)
x = jnp.ones([10, 20])
z = layer(params, x)
assert z.dtype == full

print(jax.make_jaxpr(layer)(params, x))



{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }
{ lambda ; a:f16[20,30] b:f16[30] c:f16[30,40] d:f16[40] e:f32[10,20]. let
    f:f16[10,20] = convert_element_type[new_dtype=float16 weak_type=False] e
    g:f16[10,30] = dot_general[dimension_numbers=(([1], [0]), ([], []))] f a
    h:f16[1,30] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 30)] b
    i:f16[10,30] = add g h
    j:f32[10,30] = convert_element_type[new_dtype=float32 weak_type=False] i
    k:f32[30,40] = convert_element_type[new_dtype=float32 weak_type=False] c
    l:f32[40] = convert_element_type[new_dtype=float32 weak_type=False] d
    m:f32[10,40] = dot_general[dimension_numbers=(([1], [0]), ([], []))] j k
    n:f32[1,40] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 40)] l
    o:f32[10,40] = add m n
  in (o,) }
